# Experiments

In this notebook, the different models are trained and the necessary data are recorded.


In [ ]:
import torch

%load_ext autoreload
%autoreload 2

from main import experiments

In this section we proceed to train the scalable model for the different scales.

We fix the different parameters :
- Batch size : 128
- Scales ```torch.linspace(1.0/9, 120 - 1/9, 50)```, corresponding to 1'400 up to 2'806'751 parameters.
- Initial learning rate : $10^{-3}$
- $\beta_1=0.9$ and $\beta_2 = 0.999$, numerical stability variable $\varepsilon = 10^{-8}$
- 50 epochs
- MNIST dataset
- Use learning rate scheduler
- 10 experiments (i.e. 10 training of the same model) for each optimizer and each scale. The seed is incremented after each experiment.

Important note : before running the experiments, you need to create the following folders :
- ```./results```
- ```./results/weights```

Not doing so will result in an error when trying to save the data.

In [ ]:
# Configuration for ZO-AdaMM
config = {
    "seed": 42,
    "batch_size": 128,
    "net": "scalable",
    "scale": 1.0/9,
    "opt_params": [1e-3, 0.9, 0.999, 1e-8],
    "optimizer": 'ZO-AdaMM',
    "epochs": 50,
    "dataset": "mnist",
    "zo_optim": True,
    "mu": 1e-3,
    "use_scheduler": True,
    "verbose": True
}

In [ ]:
# Define the different scales of the model
scales = torch.linspace(1.0/9, 120 - 1/9, 50).tolist()

In [ ]:
experiments(config, 'results', scales, 10, record_weights=True, weights_path=f'results/weights/weights_sequence_zo_adamm')

In [ ]:
# Configuration for AdaMM
config = {
    "seed": 42,
    "batch_size": 128,
    "net": "scalable",
    "scale": 1.0/9,
    "opt_params": [1e-3, 0.9, 0.999, 1e-8],
    "optimizer": 'AdaMM',
    "epochs": 50,
    "dataset": "mnist",
    "zo_optim": False,
    "mu": 1e-3,
    "use_scheduler": True,
    "verbose": True
}

In [ ]:
experiments(config, 'results', scales, 10, record_weights=True, weights_path=f'results/weights/weights_sequence_adamm')